# Chapter 10 - Data Aggregation and Group Operations

## 10.1 GroupBy Mechanics

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randn(5),
                  'data2': np.random.randn(5)})

df

,key1,key2,data1,data2
0,a,one,-3.790209,0.505249
1,a,two,-1.670026,-0.083977
2,b,one,-0.429554,0.764432
3,b,two,-0.187452,-1.078723
4,a,one,0.399283,-2.283111


In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped

In [6]:
grouped.mean()

key1
a   -1.686984
b   -0.308503
Name: data1, dtype: float64

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [8]:
means

key1  key2
a     one    -1.695463
      two    -1.670026
b     one    -0.429554
      two    -0.187452
Name: data1, dtype: float64

In [9]:
means.unstack()

key2,one,two
key1,,
a,-1.695463,-1.670026
b,-0.429554,-0.187452


In [10]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -1.670026
            2006   -0.429554
Ohio        2005   -1.988831
            2006    0.399283
Name: data1, dtype: float64

In [11]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,-1.686984,-0.620613
b,-0.308503,-0.157145


In [13]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one  -1.695463 -0.888931
     two  -1.670026 -0.083977
b    one  -0.429554  0.764432
     two  -0.187452 -1.078723

In [14]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### Iterating Over Groups

In [15]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one -3.790209  0.505249
1    a  two -1.670026 -0.083977
4    a  one  0.399283 -2.283111
b
  key1 key2     data1     data2
2    b  one -0.429554  0.764432
3    b  two -0.187452 -1.078723


In [16]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one -3.790209  0.505249
4    a  one  0.399283 -2.283111
a two
  key1 key2     data1     data2
1    a  two -1.670026 -0.083977
b one
  key1 key2     data1     data2
2    b  one -0.429554  0.764432
b two
  key1 key2     data1     data2
3    b  two -0.187452 -1.078723


In [17]:
pieces = dict(list(df.groupby('key1')))

pieces['b']

,key1,key2,data1,data2
2,b,one,-0.429554,0.764432
3,b,two,-0.187452,-1.078723


In [18]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [19]:
grouped = df.groupby(df.dtypes, axis=1)

In [20]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0 -3.790209  0.505249
1 -1.670026 -0.083977
2 -0.429554  0.764432
3 -0.187452 -1.078723
4  0.399283 -2.283111
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### Selecting a Column or Subset of Columns

In [21]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.888931
     two  -0.083977
b    one   0.764432
     two  -1.078723

In [22]:
s_grouped = df.groupby(['key1', 'key2'])['data2']

s_grouped

In [23]:
s_grouped.mean()

key1  key2
a     one    -0.888931
      two    -0.083977
b     one     0.764432
      two    -1.078723
Name: data2, dtype: float64

### Grouping with Dicts and Series

In [24]:
people = pd.DataFrame(np.random.randn(5, 5),
                     columns=['a', 'b', 'c', 'd', 'e'],
                     index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

people.iloc[2:3, [1,2]] = np.nan

people

,a,b,c,d,e
Joe,1.316194,-0.562782,1.153359,1.499400,0.733204
Steve,0.050224,0.319801,0.793410,-1.094009,0.742165
Wes,1.531036,NaN,NaN,0.362621,-1.069474
Jim,-0.694690,0.361547,0.294286,-0.109282,-0.142491
Travis,-0.001835,-0.903514,0.770282,-0.073948,1.110231


In [25]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
          'd': 'blue', 'e': 'red', 'f': 'orange'}

In [26]:
by_column = people.groupby(mapping, axis=1)

In [27]:
by_column.sum()

,blue,red
Joe,2.652759,1.486616
Steve,-0.300598,1.112190
Wes,0.362621,0.461562
Jim,0.185003,-0.475634
Travis,0.696334,0.204882


In [28]:
map_series = pd.Series(mapping)

In [29]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [30]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### Grouping with Functions

In [31]:
people.groupby(len).sum()

,a,b,c,d,e
3,2.152540,-0.201235,1.447644,1.752738,-0.478760
5,0.050224,0.319801,0.793410,-1.094009,0.742165
6,-0.001835,-0.903514,0.770282,-0.073948,1.110231


In [36]:
key_list = ['one', 'one', 'one', 'two', 'two']

people.groupby([len, key_list]).min()

a         b         c         d         e
3 one  1.316194 -0.562782  1.153359  0.362621 -1.069474
  two -0.694690  0.361547  0.294286 -0.109282 -0.142491
5 one  0.050224  0.319801  0.793410 -1.094009  0.742165
6 two -0.001835 -0.903514  0.770282 -0.073948  1.110231

### Grouping by Index Levels

In [37]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                   names=['cty', 'tenor'])

hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -2.181388  0.272604  1.795656 -0.413562 -1.193558
1     -1.143427 -2.949846  0.296888  0.892117  1.087466
2     -1.005023  0.033446 -0.929243  1.757029  0.618327
3     -0.950445 -1.077432  0.231126 -0.993228  1.731599

In [38]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation